# Read Jsons Data (Batch)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#df = spark.read.format("json")\
#            .option("inferShcema",True)\
 #           .option("multiline",True)\
  #          .load("/Volumes/workspace/stream/streaming/jsonsource/")

#display(df)

customer,items,metadata,order_id,payment,timestamp
"List(List(Toronto, Canada, M5H 2N2), 501, john@example.com, John Doe)","List(List(I100, 25.99, Wireless Mouse, 2), List(I101, 15.49, USB-C Adapter, 1))","List(List(campaign, back_to_school), List(channel, email))",ORD1001,"List(Credit Card, TXN7890)",2025-06-01T10:15:00Z


In [0]:
df.printSchema()

root
 |-- customer: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- postal_code: string (nullable = true)
 |    |-- customer_id: long (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- quantity: long (nullable = true)
 |-- metadata: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- payment: struct (nullable = true)
 |    |-- method: string (nullable = true)
 |    |-- transaction_id: string (nullable = true)


In [0]:
#df = df.select("order_id", "timestamp","customer.customer_id","customer.name","customer.email","customer.address.city","customer.address.country","customer.address.postal_code", "items", "payment.method", "payment.transaction_id", "metadata")

#df = df.withColumn("items", explode_outer("items"))
#display(df)


order_id,timestamp,customer_id,name,email,city,country,postal_code,items,method,transaction_id,metadata
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,"List(I100, 25.99, Wireless Mouse, 2)",Credit Card,TXN7890,"List(List(campaign, back_to_school), List(channel, email))"
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,"List(I101, 15.49, USB-C Adapter, 1)",Credit Card,TXN7890,"List(List(campaign, back_to_school), List(channel, email))"


In [0]:
#df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "items.item_id", "items.price", "items.product_name", "items.quantity", "method", "transaction_id", "metadata")

#display(df)

order_id,timestamp,customer_id,name,email,city,country,postal_code,item_id,price,product_name,quantity,method,transaction_id,metadata
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,"List(List(campaign, back_to_school), List(channel, email))"
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,"List(List(campaign, back_to_school), List(channel, email))"


In [0]:
#df =df.withColumn("metadata", explode_outer("metadata"))
#display(df)

order_id,timestamp,customer_id,name,email,city,country,postal_code,item_id,price,product_name,quantity,method,transaction_id,metadata
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,"List(campaign, back_to_school)"
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,"List(channel, email)"
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,"List(campaign, back_to_school)"
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,"List(channel, email)"


In [0]:
#df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "item_id", "price", "product_name", "quantity", "method", "transaction_id", "metadata.key", "metadata.value")

#display(df)

order_id,timestamp,customer_id,name,email,city,country,postal_code,item_id,price,product_name,quantity,method,transaction_id,key,value
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,campaign,back_to_school
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,channel,email
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,campaign,back_to_school
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,channel,email


# Read Streaming Data

In [0]:
my_shcema = """ order_id STRING,
            timestamp STRING,
            customer STRUCT<
                customer_id: INT,
                name: STRING,
                email: STRING,
                address: STRUCT<
                    city: STRING,
                    postal_code: STRING,
                    country: STRING
                >
            >,
            items ARRAY<STRUCT<
                item_id: STRING,
                product_name: STRING,
                quantity: INT,
                price: DOUBLE
            >>,
            payment STRUCT<
                method: STRING,
                transaction_id: STRING
            >,
            metadata ARRAY<STRUCT<
            key: STRING,
            value: STRING
            >>"""

In [0]:
#Shcema Inference for streaminging
#spark.conf.set("spark.sql.streaming.schemaInference", True)

df = spark.readStream.format("json")\
            .option("multiline",True)\
            .schema(my_shcema)\
            .load("/Volumes/workspace/stream/streaming/jsonsource/")

#Transformations 

df = df.select("order_id", "timestamp","customer.customer_id","customer.name","customer.email","customer.address.city","customer.address.country","customer.address.postal_code", "items", "payment.method", "payment.transaction_id", "metadata")

df =df.withColumn("items", explode_outer("items"))

df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "items.item_id", "items.price", "items.product_name", "items.quantity", "method", "transaction_id", "metadata")

df =df.withColumn("metadata", explode_outer("metadata"))

df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "item_id", "price", "product_name", "quantity", "method", "transaction_id", "metadata.key", "metadata.value")



In [0]:
df.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("checkpointLocation", "/Volumes/workspace/stream/streaming/jsonsink/checkpoint")\
    .option("path", "/Volumes/workspace/stream/streaming/jsonsink/dData")\
    .start()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/stream/streaming/jsonsink/dData/`

order_id,timestamp,customer_id,name,email,city,country,postal_code,item_id,price,product_name,quantity,method,transaction_id,key,value
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,campaign,back_to_school
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I100,25.99,Wireless Mouse,2,Credit Card,TXN7890,channel,email
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,campaign,back_to_school
ORD1001,2025-06-01T10:15:00Z,501,John Doe,john@example.com,Toronto,Canada,M5H 2N2,I101,15.49,USB-C Adapter,1,Credit Card,TXN7890,channel,email
ORD1002,2025-06-01T10:30:00Z,502,Alice Smith,alice@example.com,Vancouver,Canada,V5K 0A1,I102,45.0,Bluetooth Keyboard,1,PayPal,TXN7891,campaign,cyber_monday
ORD1002,2025-06-01T10:30:00Z,502,Alice Smith,alice@example.com,Vancouver,Canada,V5K 0A1,I102,45.0,Bluetooth Keyboard,1,PayPal,TXN7891,channel,affiliate


# Archiving

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/stream/streaming/jsonsourcesinknew")

True

In [0]:
df = spark.readStream.format("json")\
            .option("multiline",True)\
            .schema(my_shcema)\
            .option("cleanSource", "archive")\
            .option("sourceAchiveDir","/Volumes/workspace/stream/streaming/jsonsourceArchive")\
            .load("/Volumes/workspace/stream/streaming/jsonsourcenew")

#Transformations 

df = df.select("order_id", "timestamp","customer.customer_id","customer.name","customer.email","customer.address.city","customer.address.country","customer.address.postal_code", "items", "payment.method", "payment.transaction_id", "metadata")

df =df.withColumn("items", explode_outer("items"))

df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "items.item_id", "items.price", "items.product_name", "items.quantity", "method", "transaction_id", "metadata")

df =df.withColumn("metadata", explode_outer("metadata"))

df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "item_id", "price", "product_name", "quantity", "method", "transaction_id", "metadata.key", "metadata.value")


In [0]:
df = spark.readStream.format("json")\
    .option("multiline", True)\
    .schema(my_shcema)\
    .option("cleanSource", "archive")\
    .option("sourceArchiveDir", "/Volumes/workspace/stream/streaming/jsonsourceArchive")\
    .load("/Volumes/workspace/stream/streaming/jsonsourcenew")

# Transformations
df = df.select("order_id", "timestamp","customer.customer_id","customer.name","customer.email","customer.address.city","customer.address.country","customer.address.postal_code", "items", "payment.method", "payment.transaction_id", "metadata")

df = df.withColumn("items", explode_outer("items"))

df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "items.item_id", "items.price", "items.product_name", "items.quantity", "method", "transaction_id", "metadata")

df = df.withColumn("metadata", explode_outer("metadata"))

df = df.select("order_id", "timestamp", "customer_id", "name", "email", "city", "country", "postal_code", "item_id", "price", "product_name", "quantity", "method", "transaction_id", "metadata.key", "metadata.value")


In [0]:
df.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("path", "/Volumes/workspace/stream/streaming/delta_output")\
    .option("checkpointLocation", "/Volumes/workspace/stream/streaming/jsonsourcesinknew/checkpoint")\
    .start()